In [1]:
!pip install torchmetrics
!pip install torch-fidelity
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 24.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torchvision
import torchmetrics
import torch_fidelity
from torch import nn
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from collections import OrderedDict
import numpy as np
from tqdm.auto import tqdm
from torchmetrics.image.fid import FrechetInceptionDistance
from PIL import Image
import torchvision.transforms as T

In [21]:
trafos = torchvision.transforms.Compose([
    torchvision.transforms.Grayscale(num_output_channels=3),
    torchvision.transforms.ToTensor(),
])
test_dataset = datasets.MNIST(root='mnist/', train=False,download=True, transform=trafos)

In [22]:
test_dataset[0][0].shape

torch.Size([3, 28, 28])

In [23]:
images = []
count = 0

for (image, label) in test_dataset:
  count += 1
  images.append(image)

  if count >= 5040:
    break

real_images = torch.cat([image.unsqueeze(0) for image in images])
print(real_images.shape)

torch.Size([5040, 3, 28, 28])


In [7]:
ddpm = torch.load('annotated_tests_ddpm.pt')
ddim = torch.load('annotated_tests_ddim.pt')

In [8]:
class AugmentedMNIST(Dataset):
  def __init__(self, tests):
    self.samples = tests
  
  def __len__(self):
    return len(self.samples)
  
  def __getitem__(self, idx):
    return self.samples[idx]

In [18]:
ddpm_tests = AugmentedMNIST(ddpm)
ddim_tests = AugmentedMNIST(ddim)

In [19]:
images = []
for (img, label) in ddim_tests:
  images.append(torch.cat([img, img, img], dim=0))

fake_ddim_images = torch.cat([image.unsqueeze(0) for image in images])

In [20]:
fake_ddim_images.shape

torch.Size([5040, 3, 28, 28])

In [24]:
images = []
for (img, label) in ddpm_tests:
  images.append(torch.cat([img, img, img], dim=0))

fake_ddpm_images = torch.cat([image.unsqueeze(0) for image in images])

In [25]:
fake_ddpm_images.shape

torch.Size([5040, 3, 28, 28])

In [26]:
fid = FrechetInceptionDistance(normalize=True)
fid.update(real_images, real=True)
fid.update(fake_ddpm_images, real=False)

In [27]:
fid.compute()

tensor(20.0895)

In [28]:
fid = FrechetInceptionDistance(normalize=True)
fid.update(real_images, real=True)
fid.update(fake_ddim_images, real=False)

In [29]:
fid.compute()

tensor(49.5054)